In [1]:
# # Polash Deb
# # EE 5323 Digital Image Processing
# # Project
# # Date: 05/01/2023



# importing different libraries
import cv2
import random
import numpy as np
from PIL import Image,ImageChops
from skimage.util import random_noise
import scipy.ndimage as ndimage
from matplotlib import pyplot as plt
from skimage import io, restoration,filters,img_as_float,color
from skimage.restoration import denoise_nl_means,denoise_tv_bregman,denoise_tv_chambolle

# user to enter the name of the image file
image = input("Enter the name of the image file with extension (Ex: lena.jpg): ")

# reading the input image
img=cv2.imread(image)

# Splitting RGB image
B,G,R= cv2.split(img)

# user to choose a filter
filter_name = input("Choose a filter: \n1. Arithmetic Mean \n2. Median \n3. Minimum \n4. Maximum \n5. Geometric Mean \n6. Harmonic Mean \n7. Contraharmonic Mean \nEnter filter number(Please enter any value from 1-7): ")
filter_number = int(filter_name)


# Apply the selected filter to each color channel

if filter_number == 1:    # Arithmetic Mean Filter
    #apply cv2.blur function to each color channel
    b = cv2.blur(B, (5, 5))
    g = cv2.blur(G, (5, 5))
    r = cv2.blur(R, (5, 5))
    
elif filter_number == 2:    # Median Filter
    #apply cv2.medianBlur function to each color channel
    b = cv2.medianBlur(B, 5)
    g = cv2.medianBlur(G, 5)
    r = cv2.medianBlur(R, 5)
    
elif filter_number == 3:    # Minimum Filter
    #apply cv2.erode function to each color channel
    b = cv2.erode(B, np.ones((5, 5)))
    g = cv2.erode(G, np.ones((5, 5)))
    r= cv2.erode(R, np.ones((5, 5)))
    
elif filter_number == 4:    # Maximum Filter
    #apply cv2.dilate function to each color channel
    b = cv2.dilate(B, np.ones((5, 5)))
    g = cv2.dilate(G, np.ones((5, 5)))
    r = cv2.dilate(R, np.ones((5, 5)))
    
elif filter_number == 5:    # Geometric mean Filter
    #define a function for Geometric Mean
    def geometric_mean(img,x):
        rows, cols = img.shape[:2]
        a = 5
        b = int((a-1)/2)
        pad_img = cv2.copyMakeBorder(img, *[b]*4, cv2.BORDER_DEFAULT)
        geomean = np.zeros_like(img)
        for r in range(rows):
            for c in range(cols):
                geomean[r, c] = np.prod(pad_img[r:r+a, c:c+a])**(1/(a**2))
        geomean = np.uint8(geomean)
        # Normalize the output
        img2=cv2.normalize(geomean, None, 0, 255, cv2.NORM_MINMAX,dtype=cv2.CV_8UC1)
        return img2
    
    #apply geometric_mean function in each color channel
    b = geometric_mean(B,5)
    g = geometric_mean(G,5)
    r = geometric_mean(R,5)
    
elif filter_number == 6:    # Harmonic Mean Filter
    b= (len(B)/np.sum(1/np.array(B)))
    g= (len(G)/np.sum(1/np.array(G)))
    r= (len(R)/np.sum(1/np.array(R)))

elif filter_number == 7:    # Contraharmonic Mean Filter
    b=np.sum(np.square(np.array(B)))/np.sum(np.array(B))
    g=np.sum(np.square(np.array(G)))/np.sum(np.array(G))
    r=np.sum(np.square(np.array(R)))/np.sum(np.array(R))
    

# Combine r, b and g channel together
filter_output = np.dstack((b,g,r))

#saving Filtered image
cv2.imwrite('Filter Image.png', filter_output)

# Display the filtered images
cv2.imshow('Filtered Image', filter_output)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)



# user to choose a noise
print("\n")
noise_name = input("Choose a noise: \n1. White noise \n2. Uniform noise \n3. “Salt and Pepper” noise \n4. Gaussian Noise \n5. Rayleigh Noise \n6. Erlang Noise \n7. Exponential Noise \n8. Sinusoidal Noise \nEnter noise number(Please enter any value from 1-8): ")
noise_number = int(noise_name)

# apply noise to original image

if noise_number == 1:   # white Noise
    # Get the magnitude from the user
    mag = float(input("Enter the magnitude(it should be less than 1(range 0-1))(value near to 1 means excessive noise): "))
    
    # Generate white noise using salt noise and add to original image
    noise = random_noise(img, mode='salt', amount=mag)
    
    # Normalize the image range from 0 to 255
    img2=cv2.normalize(noise, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    
    x=1
    
elif noise_number == 2:   # Uniform Noise
    # Get the range for the noise from the user
    low = float(input("Enter the noise range's lowest value(it should not be less than 0): "))
    high = float(input("Enter the noise range's highest value.(it should be less than 255): "))
    
    # Generate uniform noise with the same size as the image
    noise = np.random.uniform(low, high, img.shape)
    
    # Add the noise to the each color channel
    b1 = cv2.add(B, noise[:,:,0].astype(np.uint8))
    g1 = cv2.add(G, noise[:,:,1].astype(np.uint8))
    r1 = cv2.add(R, noise[:,:,2].astype(np.uint8))
    
    x=0
    
elif noise_number == 3:   # Salt & Pepper Noise    
    
    # Get the magnitude and bandwidth from the user
    mag = float(input("Enter the magnitude(it should be less than 1(range 0-1))(value near to 1 means excessive noise): "))
    prob = float(input("Enter the probability(it should be less than 1(range 0-1)): "))
    
    # Generate Salt & Pepper noise and add to original image
    noise = random_noise(img, mode='s&p', amount=mag, salt_vs_pepper=prob)
    
    # Normalize the image range from 0 to 255
    img2=cv2.normalize(noise, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    
    x=1

elif noise_number == 4:   # Gaussian Noise 
    # Get the magnitude from the user
    m = float(input("Enter the mean(it should be near to 0 (range 0-1)): "))
    v = float(input("Enter the variance(it should be less than 1(range 0-1)): "))
    
    # Generate Gaussian noise and add to original image
    noise = random_noise(img, mode='gaussian', mean=m,var=v)
    
    # Normalize the image range from 0 to 255
    img2=cv2.normalize(noise, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    
    x=1 
    
elif noise_number==5: #Rayleigh Noise
    
    # Convert the image to a numpy array and normalize pixel values
    img2 = np.array(img, dtype=np.float32) / 255.0

    # Generate Rayleigh noise with the same dimensions as the image
    noise = np.random.rayleigh(scale=0.2, size=img2.shape[:2])

    # Add the noise to each channel of the image
    img2[:,:,0] += noise
    img2[:,:,1] += noise
    img2[:,:,2] += noise

    # Clip the pixel values to ensure they remain in the range of 0 to 1
    img2 = np.clip(img2, 0.0, 1.0)

    # Convert the image back to uint8 data type
    img2 = (img2 * 255.0).astype(np.uint8)
    
    x=1

elif noise_number==6: # Erlang Noise   
    
    # Convert the image to a numpy array and normalize pixel values
    img2 = np.array(img, dtype=np.float32) / 255.0
    
    # Generate Erlang noise with the same dimensions as the image
    noise = np.random.gamma(2, 0.1, size=img2.shape[:2])
    
    # Add the noise to each channel of the image
    img2[:,:,0] += noise
    img2[:,:,1] += noise
    img2[:,:,2] += noise

    # Clip the pixel values to ensure they remain in the range of 0 to 1
    img2 = np.clip(img2, 0.0, 1.0)

    # Convert the image back to uint8 data type
    img2 = (img2 * 255.0).astype(np.uint8)
    
    x=1

elif noise_number==7: # Exponential Noise   
    
    # Convert the image to a numpy array and normalize pixel values
    img2 = np.array(img, dtype=np.float32) / 255.0
    
    # Generate Exponential noise with the same dimensions as the image
    noise = np.random.exponential(2, size=img2.shape[:2])
    
    # Multiply the noise with a scaling factor to control the noise intensity
    noise *= 0.1

    # Add the noise to each channel of the image
    img2[:,:,0] += noise
    img2[:,:,1] += noise
    img2[:,:,2] += noise

    # Clip the pixel values to ensure they remain in the range of 0 to 1
    img2 = np.clip(img2, 0.0, 1.0)

    # Convert the image back to uint8 data type
    img2 = (img2 * 255.0).astype(np.uint8)
    
    x=1
    
elif noise_number==8: # Sinusoidal Noise      
    # read the original image
    img2=cv2.imread(image)
    
    #calculate number of row and column of original image
    h, w = img2.shape[:2]
   
    #create array value from 0 to 2 Pi
    a=np.linspace(0,2*np.pi,h)
    b=np.linspace(0,2*np.pi,w)
    
    #create a mashgrid
    a, b = np.meshgrid(np.arange(w), np.arange(h))

    # Set amplitude and frequency of sinusoidal noise
    amplitude = 25  # amplitude
    frequency = 100  # frequency

    # Generate sinusoidal noise
    noise = amplitude * np.sin((frequency * a)+ (frequency * b))

    # Add the noise to each channel of the image
    img2[:,:,0] += noise.astype(np.uint8)
    img2[:,:,1] += noise.astype(np.uint8)
    img2[:,:,2] += noise.astype(np.uint8)

    x=1  
    
if x==0:
    # Combine r1, b1 and g1 together
    noisy_output = np.dstack((b1,g1,r1))
elif x==1:
    noisy_output = img2    

#saving noisy image
cv2.imwrite('Noisy Image.png', noisy_output)    
    
# Display the noisy image
cv2.imshow('Noisy Image', noisy_output)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)    


# user to choose a filter for removing noise
print("\n")
noise_f_name = input("Choose a noise which want to remove: \n1. White noise \n2. Uniform noise \n3. Gaussian Noise \n4. Rayleigh Noise \n5. Erlang Noise \n6. Exponential Noise \nEnter the number of noise which you want to remove(Please enter any value from 1-6): ")
noise_f_number = int(noise_f_name)

# apply filter to noisy image

if noise_f_number==1: # White Noise
    # Get the kernel size for filter from the user
    x=int(input("Enter kernel size for filter(it should be odd number): "))
    
    # Apply Median Filter to remove White noise
    noisy_filtered_output = cv2.medianBlur(noisy_output, x)
    

elif noise_f_number==2: # Uniform Noise
    # Get the filter strenght value from the user
    x = float(input("Enter the filter strength(higher value makes the image blurry and also increase the run time): "))
    y = float(input("Enter the filter strength for color components(higher value makes the image blurry and also increase the run time): "))
    
    # Apply fastNlMeansDenoisingColored function to remove uniform noise
    denoise = cv2.fastNlMeansDenoisingColored(noisy_output, None, x, y, 60, 21)

    # Apply the low pass filter to the image
    noisy_filtered_output = cv2.GaussianBlur(denoise, (5, 5), 0)

elif noise_f_number==3: # Gaussian Noise
    
    # Apply Gaussian filter to remove Gaussian noise
    noisy_filtered_output = filters.gaussian(noisy_output, sigma=2.5, multichannel=True)
    
elif noise_f_number==4: # Rayleigh Noise
    #convert noisy image into a float
    y = img_as_float(noisy_output)
    
    #apply denoise_nl_means function to the noisy image where h parameter controls the strength of the filtering
    noisy_filtered_output = denoise_nl_means(y, h=0.1, sigma=0.05, fast_mode=True, multichannel=True)

elif noise_f_number==5: # Erlang Noise
    #convert noisy image into a float
    y = img_as_float(noisy_output)
    
    #Apply the denoise_tv_bregman function to the noisy image and weight parameter control the filtering strenght(Smaller is better)
    noisy_filtered_output = denoise_tv_bregman(y, weight=0.8, max_iter=100, eps=0.0001, isotropic=True, multichannel=True)

elif noise_f_number==6: # Exponential Noise 
    #convert noisy image into a float
    y = img_as_float(noisy_output)
    
    #Apply the denoise_tv_chambolle function to the noisy image and weight parameter control the filtering strenght(higher is better)
    noisy_filtered_output = denoise_tv_chambolle(y, weight=0.4, eps=0.0001, n_iter_max=200, multichannel=True)
    
#Normalize the output and saving filtered image
ab=cv2.normalize(noisy_filtered_output, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
cv2.imwrite('Noisy Filtered Image.png', ab)    
    
#Display the filtered image
cv2.imshow('Noisy Filtered Image', noisy_filtered_output)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


# user to choose a color adjustment
print("\n")
color_name = input("Choose a color adjustment: \n1. Color Negative \n2. Convert to CMY \n3. Display of Red, Green, and Blue planes separately \n4. Display of Cyan, Magenta, and Yellow planes separately \n5. Display of Y, Cb, and Cr planes separately \n6. Display of H, S, and V planes separately \n7. Individual adjustments of R, G, and B components \nEnter the number of color adjustment(Please enter any value from 1-7): ")
color_number = int(color_name)


# apply color adjustment to original image

if color_number==1: # Color negative
    # Convert the image to different color spaces
    image_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Split each color space into individual channels
    h, s, v = cv2.split(image_hsv)
    
    # Create a negative image by swapping channels from different color spaces
    n_image = cv2.merge((s, s, s))
    
    # Save the negative image
    cv2.imwrite('Negative Image.png', n_image)
    
    # display the negative image
    cv2.imshow('Negative Image', n_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

elif color_number==2: # Covert to CMY
    # Calculating C,M and Y
    C = 1 - R
    M = 1 - G
    Y = 1 - B
    
    # Combine C, M and Y together
    cmy = np.dstack((C,M,Y))
    
    # Save the CMY image
    cv2.imwrite('CMY Image.png', cmy)
    
    # display the CMY image
    cv2.imshow('CMY Image', cmy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

elif color_number==3: # Display of Red, Green, and Blue planes separately
    #keeping blue channel value and making both green and red channel value zero
    blue = img.copy()
    blue[:, :, 1] = 0
    blue[:, :, 2] = 0
    
    #keeping green channel value and making both red and blue channel value zero
    green = img.copy()
    green[:, :, 0] = 0
    green[:, :, 2] = 0
    
    #keeping red channel value and making both blue and green channel value zero
    red = img.copy()
    red[:, :, 0] = 0
    red[:, :, 1] = 0
    
    
    # Save the Red, Green & Blue image separately
    cv2.imwrite('RED Image.png', red)
    cv2.imwrite('GREEN Image.png', green)
    cv2.imwrite('BLUE Image.png', blue)
    
    # display the Red, Green & Blue image separately
    cv2.imshow('RED Image', red)
    cv2.waitKey(0)
    cv2.imshow('GREEN Image', green)
    cv2.waitKey(0)
    cv2.imshow('BLUE Image', blue)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
 
elif color_number==4: # Display of Cyan, Magenta, and Yellow planes separately   
    # Calculating C,M and Y
    C = 1 - R
    M = 1 - G
    Y = 1 - B
    
    # Save the Cyan, Magenta & Yellow image separately
    cv2.imwrite('CYAN Image.png', C)
    cv2.imwrite('MAGENTA Image.png', M)
    cv2.imwrite('YELLOW Image.png', Y)
    
    # display the Cyan, Magenta & Yellow image separately
    cv2.imshow('CYAN Image', C)
    cv2.waitKey(0)
    cv2.imshow('MAGENTA Image', M)
    cv2.waitKey(0)
    cv2.imshow('YELLOW Image', Y)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
elif color_number==5: # Display of Y, Cb, and Cr planes separately
    # Calculating Y,Cb and Cr
    Y = np.uint8(np.round(0.299*R + 0.587*G + 0.114*B))
    Cb = np.uint8(np.round( -0.1687*R - 0.3313*G + 0.5*B + 128))
    Cr = np.uint8(np.round( 0.5*R - 0.4187*G - 0.0813*B + 128))

    # Combine Y, Cb and Cr together
    YCbCr = np.dstack((Y,Cb,Cr))
    
    # Save the YCbCr,Y,Cb and Cr image separately
    cv2.imwrite('YCbCr.png', YCbCr)
    cv2.imwrite('Luminance.png', Y)
    cv2.imwrite('Chrominance-blue.png', Cb)
    cv2.imwrite('Chrominance-red.png', Cr)
    
    # display the YCbCr,Y,Cb and Cr image separately
    cv2.imshow('YCbCr Image', YCbCr)
    cv2.waitKey(0)
    cv2.imshow('Luminance Image', Y)
    cv2.waitKey(0)
    cv2.imshow('Chrominance-blue Image', Cb)
    cv2.waitKey(0)
    cv2.imshow('Chrominance-red Image', Cr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    
elif color_number==6: # Display of H, S, and V planes separately    
    #convert original image to HSV image
    HSV=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Splitting H, S and V channel
    H,S,V=cv2.split(HSV)
    
    # Save the HSV,H,S and V image separately
    cv2.imwrite('HSI.png', HSV)
    cv2.imwrite('Hue.png', H)
    cv2.imwrite('Saturation.png', S)
    cv2.imwrite('Intensity.png', V)
    
    # display the HSV,H,S and V image separately
    cv2.imshow('HSI Image', HSV)
    cv2.waitKey(0)
    cv2.imshow('Hue Image', H)
    cv2.waitKey(0)
    cv2.imshow('Saturation Image', S)
    cv2.waitKey(0)
    cv2.imshow('Intensity Image', V)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

elif color_number==7: # Individual adjustments of R, G, and B components
    #taking input value of RED from user
    print("\n")
    r =float(input("Please enter any value of Red from -255 to 255: "))
    
    #Adjust value RED
    if r>=0 and r<=255:
        r1=cv2.add(R,r)
    else:
        r1=cv2.subtract(R,r)
    
    #taking input value of GREEN from user
    print("\n")
    g =float(input("Please enter any value of Green from -255 to 255: "))
    
    #Adjust value GREEN
    if g>=0 and g<=255:
        g1=cv2.add(G,g)
    else:
        g1=cv2.subtract(G,g)
        
    #taking input value of BLUE from user
    print("\n")
    b =float(input("Please enter any value of Blue from -255 to 255: "))
    
    #Adjust value BLUE
    if b>=0 and b<=255:
        b1=cv2.add(B,b)
    else:
        b1=cv2.subtract(B,b)
        
    #Combine all channels together
    new_rgb=np.dstack((b1,g1,r1))
    
    # Save the New RGB image
    cv2.imwrite('New_RGB.png', new_rgb)
    
    # display the New RGB image
    cv2.imshow('New_RGB Image', new_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)


Enter the name of the image file with extension (Ex: lena.jpg): lena_color.png
Choose a filter: 
1. Arithmetic Mean 
2. Median 
3. Minimum 
4. Maximum 
5. Geometric Mean 
6. Harmonic Mean 
7. Contraharmonic Mean 
Enter filter number(Please enter any value from 1-7): 1


Choose a noise: 
1. White noise 
2. Uniform noise 
3. “Salt and Pepper” noise 
4. Gaussian Noise 
5. Rayleigh Noise 
6. Erlang Noise 
7. Exponential Noise 
8. Sinusoidal Noise 
Enter noise number(Please enter any value from 1-8): 4
Enter the mean(it should be near to 0 (range 0-1)): 0.5
Enter the variance(it should be less than 1(range 0-1)): 0.5


Choose a noise which want to remove: 
1. White noise 
2. Uniform noise 
3. Gaussian Noise 
4. Rayleigh Noise 
5. Erlang Noise 
6. Exponential Noise 
Enter the number of noise which you want to remove(Please enter any value from 1-6): 3


C:\Users\deb14\AppData\Local\Temp\ipykernel_15600\2197505170.py:305: FutureWarning: `multichannel` is a deprecated argument name for `gaussian`. It will be removed in version 1.0. Please use `channel_axis` instead.
  noisy_filtered_output = filters.gaussian(noisy_output, sigma=2.5, multichannel=True)




Choose a color adjustment: 
1. Color Negative 
2. Convert to CMY 
3. Display of Red, Green, and Blue planes separately 
4. Display of Cyan, Magenta, and Yellow planes separately 
5. Display of Y, Cb, and Cr planes separately 
6. Display of H, S, and V planes separately 
7. Individual adjustments of R, G, and B components 
Enter the number of color adjustment(Please enter any value from 1-7): 1
